# MASTER - Notebook 1 - Only temporal Cleaning
### Matteo Grazioso 884055

In [10]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json
import warnings
warnings.filterwarnings('ignore')

import myfunctions as mf # Custom functions

In [11]:
# Disply all columns and all rows
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [12]:
# Find all txt files in the data folder
txt_files = mf.find_txt_files("data/raw")

# Choose a dataset from the list of txt files
selected_dataset = mf.choose_dataset(txt_files)

if selected_dataset:
    print(f"You selected the dataset {selected_dataset}")
else:
    print("No dataset selected.")

path  = selected_dataset

The following TXT files were found:
1. data/raw/0-export.txt
2. data/raw/1-validazioni.txt
3. data/raw/2-esportazioneCompleta.txt
4. data/raw/3-esportazionePasqua23.txt
You selected the dataset data/raw/0-export.txt


In [13]:
# The file contain the data of the validation of tickets in the city of public transport of Venice.

# Import the data into a dataframe of a txt file 
# path = 'data/raw/1-validazioni.txt'                     # Period: 2022-05-13 to 2022-07-15
# path = 'data/raw/2-esportazioneCompleta.txt'            # Period: 2023-01-23 to 2023-03-14
# path = 'data/raw/3-esportazionePasqua23.txt'            # Period: 2023-04-04 to 2023-06-03

df = pd.read_csv(path, header=0, sep='\t')
# Save the name of the file in a variable for future use extracting the name of the file from the path
file_name = path.split('/')[-1].split('.')[0]
# Remove the number and the - character from the head of the file name
file_name = file_name [file_name.find('-')+1:]


In [14]:
file_name

'export'

In [15]:
# Check the first 5 rows of the data
df.head()

,DATA_VALIDAZIONE,SERIALE,FERMATA,DESCRIZIONE,TITOLO,DESCRIZIONE_TITOLO
0,11/09/2018 00:01,65676329393943813,1392,Santa Chiara,12101,Bigl.Aut.75'Mestre/Lido-tsc
1,11/09/2018 00:01,36412970881802500,5003,Lido (S.M.E.,11109,Biglietto 72 ore Roll. Venice
2,11/09/2018 00:01,36412970881806084,5003,Lido (S.M.E.,11107,"48h-Tpl 24,90-ComVe5,10"
3,11/09/2018 00:01,36412970881805828,5003,Lido (S.M.E.,11107,"48h-Tpl 24,90-ComVe5,10"
4,11/09/2018 00:01,65676744198028549,5013,S. Zaccaria,11107,"48h-Tpl 24,90-ComVe5,10"


In [16]:
# Check the last 5 rows of the data
df.tail()

,DATA_VALIDAZIONE,SERIALE,FERMATA,DESCRIZIONE,TITOLO,DESCRIZIONE_TITOLO
4876773,11/11/2018 23:56,36120397166597124,5038,"Rialto ""D""",11149,"7gg-Tpl 43,60-ComVe16,40"
4876774,11/11/2018 23:59,65676690164248581,5049,Zattere SX,11109,Biglietto 72 ore Roll. Venice
4876775,12/11/2018 00:53,36088796950622980,510,CORSIA A5,200,bordo multip BUS urb. autisti
4876776,12/11/2018 00:53,36088796950754052,510,CORSIA A5,200,bordo multip BUS urb. autisti
4876777,12/11/2018 02:48,36088796951081732,510,CORSIA A5,200,bordo multip BUS urb. autisti


In [17]:
# Create a subset of the data with the first 10% of the rows and the last 10% of the rows
# df = df.iloc[:int(len(df)*0.1),:]
# df = df.append(df.iloc[-int(len(df)*0.1):,:])


## Explorative Data Analysis


In [18]:
# Dates and hour of the validation of the ticket are in the same column 'DATA_VALIDAZIONE'
# Split the column 'DATA_VALIDAZIONE' into two columns 'DATA' and 'ORA' and convert them to datetime format
df.insert(0, 'DATA', pd.to_datetime(df['DATA_VALIDAZIONE'].str.split(' ').str[0], format='%d/%m/%Y'))
df.insert(1, 'ORA', pd.to_datetime(df['DATA_VALIDAZIONE'].str.split(' ').str[1], format='%H:%M').dt.time)

# Drop the column 'DATA_VALIDAZIONE'
# df.drop('DATA_VALIDAZIONE', axis=1, inplace=True)

# Display the first 5 rows of the dataframe
df.head()

,DATA,ORA,DATA_VALIDAZIONE,SERIALE,FERMATA,DESCRIZIONE,TITOLO,DESCRIZIONE_TITOLO
0,2018-09-11,00:01:00,11/09/2018 00:01,65676329393943813,1392,Santa Chiara,12101,Bigl.Aut.75'Mestre/Lido-tsc
1,2018-09-11,00:01:00,11/09/2018 00:01,36412970881802500,5003,Lido (S.M.E.,11109,Biglietto 72 ore Roll. Venice
2,2018-09-11,00:01:00,11/09/2018 00:01,36412970881806084,5003,Lido (S.M.E.,11107,"48h-Tpl 24,90-ComVe5,10"
3,2018-09-11,00:01:00,11/09/2018 00:01,36412970881805828,5003,Lido (S.M.E.,11107,"48h-Tpl 24,90-ComVe5,10"
4,2018-09-11,00:01:00,11/09/2018 00:01,65676744198028549,5013,S. Zaccaria,11107,"48h-Tpl 24,90-ComVe5,10"


In [19]:
# Set the format of the timestamp
df['DATA_VALIDAZIONE'] = pd.to_datetime(df['DATA_VALIDAZIONE'], format='%d/%m/%Y %H:%M')

In [20]:
# Print the date of the first and last validation using both data and hour
print('First validation: ', df['DATA'].min(), df['ORA'].min())
print('Last validation: ', df['DATA'].max(), df['ORA'].max())

# Print the number of Serial numbers
print('Number of Serial numbers: ', df['SERIALE'].nunique())

# Print the number of validation (rows)
print('Number of validation: ', df.shape[0])

# Print the number of tickets
print('Number of tickets: ', df['DESCRIZIONE_TITOLO'].nunique())
# Print the number of titolo
print('Number of titolo: ', df['TITOLO'].nunique())
# TODO: why the number of unique TITOLO is different from the number of DESCRIZIONE_TITOLO?

# Print the number of FERMATA
print('Number of FERMATA: ', df['FERMATA'].nunique())
# Print the number of DESCRIZIONE
print('Number of DESCRIZIONE: ', df['DESCRIZIONE'].nunique())
# TODO: why the number of unique DESCRIZIONE is different from the number of FERMATA?

First validation:  2018-09-11 00:00:00 00:00:00
Last validation:  2018-11-12 00:00:00 23:59:00
Number of Serial numbers:  1751985
Number of validation:  4876778
Number of tickets:  161
Number of titolo:  163
Number of FERMATA:  1672
Number of DESCRIZIONE:  892


In [21]:
# Which is the most used ticket?
df['DESCRIZIONE_TITOLO'].value_counts().head(10)

24h-Tpl 14,90-Com.Ve5,10         1247921
75'-Tpl 6,30-ComVe1,20            670822
48h-Tpl 24,90-ComVe5,10           655150
72h-Tpl 33,40-ComVe6,60           497555
Bigl.Aut.75'Mestre/Lido-tsc       343933
7gg-Tpl 43,60-ComVe16,40          255210
75'-Tpl 12,60-ComVe2,40           197263
Biglietto 72 ore Roll. Venice     185833
72ore online no aerobus            97343
7 days online no aerobus           58437
Name: DESCRIZIONE_TITOLO, dtype: int64

In [22]:
# Which is the most frequent validation in date and hour?
# Date and hour are in two different columns; DATA_VALIDAZIONE does not exist anymore
df.groupby(['DATA', 'ORA'])['SERIALE'].count().sort_values(ascending=False).head(10)
# TODO: #4 Re-aswer the question of the most frequent validation after cleaning operations

DATA        ORA     
2018-10-31  03:49:00    433
2018-09-29  17:32:00    279
2018-10-13  15:26:00    276
2018-09-30  10:45:00    274
2018-09-29  17:31:00    270
            10:59:00    269
2018-10-05  17:27:00    262
2018-09-29  15:00:00    260
2018-10-07  15:26:00    255
2018-09-30  10:42:00    252
Name: SERIALE, dtype: int64

In [23]:
# Which is the most frequent FERMATA?
df['DESCRIZIONE'].value_counts().head(10)
# TODO: #4 Re-aswer the question of the most frequent FERMATA after cleaning operations

S. Zaccaria     571024
P.le Roma (S    384088
Burano SX pe    268820
Rialto "D"      268088
Ferrovia "B"    210685
Lido (S.M.E.    200945
Murano Faro     194918
S. Marco (Va    136302
S. MARCO (Gi    129633
Ferrovia "D"    125424
Name: DESCRIZIONE, dtype: int64

## Data Cleaning

### Useless stamps

In [24]:
# Reset the index of the df and drop the old index in order to have a new index starting from 0 to the number of rows
# It is necessary to have a new index because the groupby function has created a multi-index
df.reset_index(drop=True, inplace=True)

In [25]:
# Create a new column 'MIN_TEMPORAL_GAP' that contains the minimum temporal gap between two validations for the same serial and fermata in minutes
df = df.groupby(['SERIALE','DATA', 'DESCRIZIONE']).apply(lambda x: x.assign(MIN_TEMPORAL_GAP = x['DATA_VALIDAZIONE'].diff().dt.total_seconds()/60))

In [26]:
df.head(20)

,DATA,ORA,DATA_VALIDAZIONE,SERIALE,FERMATA,DESCRIZIONE,TITOLO,DESCRIZIONE_TITOLO,MIN_TEMPORAL_GAP
0,2018-09-11,00:01:00,2018-09-11 00:01:00,65676329393943813,1392,Santa Chiara,12101,Bigl.Aut.75'Mestre/Lido-tsc,NaN
1,2018-09-11,00:01:00,2018-09-11 00:01:00,36412970881802500,5003,Lido (S.M.E.,11109,Biglietto 72 ore Roll. Venice,NaN
2,2018-09-11,00:01:00,2018-09-11 00:01:00,36412970881806084,5003,Lido (S.M.E.,11107,"48h-Tpl 24,90-ComVe5,10",NaN
3,2018-09-11,00:01:00,2018-09-11 00:01:00,36412970881805828,5003,Lido (S.M.E.,11107,"48h-Tpl 24,90-ComVe5,10",NaN
4,2018-09-11,00:01:00,2018-09-11 00:01:00,65676744198028549,5013,S. Zaccaria,11107,"48h-Tpl 24,90-ComVe5,10",NaN
5,2018-09-11,00:01:00,2018-09-11 00:01:00,65676882990430725,5013,S. Zaccaria,11107,"48h-Tpl 24,90-ComVe5,10",NaN
6,2018-09-11,00:01:00,2018-09-11 00:01:00,65676882990430725,5013,S. Zaccaria,11107,"48h-Tpl 24,90-ComVe5,10",0.0
7,2018-09-11,00:01:00,2018-09-11 00:01:00,65676882990430725,5013,S. Zaccaria,11107,"48h-Tpl 24,90-ComVe5,10",0.0
8,2018-09-11,00:01:00,2018-09-11 00:01:00,65676348342500357,5022,Giudecca Pal,11105,"24h-Tpl 14,90-Com.Ve5,10",NaN
9,2018-09-11,00:02:00,2018-09-11 00:02:00,65676348342501125,5022,Giudecca Pal,11105,"24h-Tpl 14,90-Com.Ve5,10",NaN


In [27]:
df.tail(20)

,DATA,ORA,DATA_VALIDAZIONE,SERIALE,FERMATA,DESCRIZIONE,TITOLO,DESCRIZIONE_TITOLO,MIN_TEMPORAL_GAP
4876758,2018-11-11,23:52:00,2018-11-11 23:52:00,36131496707395332,5013,S. Zaccaria,11105,"24h-Tpl 14,90-Com.Ve5,10",NaN
4876759,2018-11-11,23:52:00,2018-11-11 23:52:00,36131496708692996,5031,P.le Roma (S,11101,"75'-Tpl 6,30-ComVe1,20",NaN
4876760,2018-11-11,23:53:00,2018-11-11 23:53:00,36412971680141572,5031,P.le Roma (S,11107,"48h-Tpl 24,90-ComVe5,10",NaN
4876761,2018-11-11,23:53:00,2018-11-11 23:53:00,36131496703489540,5031,P.le Roma (S,11107,"48h-Tpl 24,90-ComVe5,10",NaN
4876762,2018-11-11,23:53:00,2018-11-11 23:53:00,36131496703489540,5031,P.le Roma (S,11107,"48h-Tpl 24,90-ComVe5,10",0.0
4876763,2018-11-11,23:53:00,2018-11-11 23:53:00,65677030511899397,5053,S. Marco (Va,11241,"7ggAerAR-Tpl55,6-CVe16,4",NaN
4876764,2018-11-11,23:53:00,2018-11-11 23:53:00,36412971680138756,5031,P.le Roma (S,11107,"48h-Tpl 24,90-ComVe5,10",NaN
4876765,2018-11-11,23:53:00,2018-11-11 23:53:00,36412971680138500,5031,P.le Roma (S,11107,"48h-Tpl 24,90-ComVe5,10",NaN
4876766,2018-11-11,23:53:00,2018-11-11 23:53:00,36412971680138756,5031,P.le Roma (S,11107,"48h-Tpl 24,90-ComVe5,10",0.0
4876767,2018-11-11,23:54:00,2018-11-11 23:54:00,65676624064772869,5013,S. Zaccaria,11108,"72h-Tpl 33,40-ComVe6,60",NaN


In [28]:
df['MIN_TEMPORAL_GAP'].value_counts()

0.0       205542
1.0        49242
2.0        32554
3.0        21626
4.0        15817
5.0        11978
6.0         9342
7.0         7299
8.0         5981
9.0         4822
10.0        3913
11.0        3164
12.0        2672
13.0        2430
14.0        1937
15.0        1783
16.0        1554
17.0        1348
18.0        1065
20.0        1018
19.0        1004
21.0         890
22.0         722
23.0         666
24.0         609
25.0         584
26.0         546
27.0         521
28.0         484
29.0         440
165.0        416
30.0         412
211.0        405
218.0        398
190.0        398
178.0        397
187.0        397
154.0        395
167.0        394
277.0        393
229.0        392
199.0        390
188.0        389
200.0        389
239.0        387
215.0        385
296.0        385
220.0        384
198.0        384
284.0        384
204.0        384
263.0        384
261.0        383
224.0        381
287.0        380
214.0        380
186.0        380
175.0        379
150.0        3

In [29]:
# How many rows have a minimum temporal gap equal to NaN?
df[df['MIN_TEMPORAL_GAP'].isna()].shape[0]

4319347

In [30]:
# Cleaning operation: remove the rows using the minimum temporal gap

# Find a reasonable delta of MIN_TEMPORAL_GAP to remove the rows that have a minimum temporal gap for the same serial and fermata less than this delta
# Print the minimum value of the column MIN_TEMPORAL_GAP
print('The minimum value of the column MIN_TEMPORAL_GAP is: {}'.format(df['MIN_TEMPORAL_GAP'].min()))

# Print the maximum value of the column MIN_TEMPORAL_GAP
print('The maximum value of the column MIN_TEMPORAL_GAP is: {}'.format(df['MIN_TEMPORAL_GAP'].max()))

# Print the mean value of the column MIN_TEMPORAL_GAP
print('The mean value of the column MIN_TEMPORAL_GAP is: {}'.format(df['MIN_TEMPORAL_GAP'].mean()))

# Print the median value of the column MIN_TEMPORAL_GAP
print('The median value of the column MIN_TEMPORAL_GAP is: {}'.format(df['MIN_TEMPORAL_GAP'].median()))

# Print the standard deviation of the column MIN_TEMPORAL_GAP
print('The standard deviation of the column MIN_TEMPORAL_GAP is: {}'.format(df['MIN_TEMPORAL_GAP'].std()))

# Print the 0.05th percentile of the column MIN_TEMPORAL_GAP
print('The 0.05th percentile of the column MIN_TEMPORAL_GAP is: {}'.format(df['MIN_TEMPORAL_GAP'].quantile(0.05)))

# Print the 0.10th percentile of the column MIN_TEMPORAL_GAP
print('The 0.10th percentile of the column MIN_TEMPORAL_GAP is: {}'.format(df['MIN_TEMPORAL_GAP'].quantile(0.10)))

# Print the 25th percentile of the column MIN_TEMPORAL_GAP
print('The 25th percentile of the column MIN_TEMPORAL_GAP is: {}'.format(df['MIN_TEMPORAL_GAP'].quantile(0.25)))

# Print the 75th percentile of the column MIN_TEMPORAL_GAP
print('The 75th percentile of the column MIN_TEMPORAL_GAP is: {}'.format(df['MIN_TEMPORAL_GAP'].quantile(0.75)))

# Print the 90th percentile of the column MIN_TEMPORAL_GAP
print('The 90th percentile of the column MIN_TEMPORAL_GAP is: {}'.format(df['MIN_TEMPORAL_GAP'].quantile(0.90)))

# Print the 95th percentile of the column MIN_TEMPORAL_GAP
print('The 95th percentile of the column MIN_TEMPORAL_GAP is: {}'.format(df['MIN_TEMPORAL_GAP'].quantile(0.95)))

# Print the 99th percentile of the column MIN_TEMPORAL_GAP
print('The 99th percentile of the column MIN_TEMPORAL_GAP is: {}'.format(df['MIN_TEMPORAL_GAP'].quantile(0.99)))

# Print the 99.9th percentile of the column MIN_TEMPORAL_GAP
print('The 99.9th percentile of the column MIN_TEMPORAL_GAP is: {}'.format(df['MIN_TEMPORAL_GAP'].quantile(0.999)))

# Decide the delta of MIN_TEMPORAL_GAP to remove the rows that have a minimum temporal gap for the same serial and fermata less than this delta
delta = df['MIN_TEMPORAL_GAP'].quantile(0.1)
if delta == 0:
    delta = df['MIN_TEMPORAL_GAP'].quantile(0.25)
if delta == 0:
    delta = df['MIN_TEMPORAL_GAP'].median()
print('The delta of MIN_TEMPORAL_GAP is: {}'.format(delta))

The minimum value of the column MIN_TEMPORAL_GAP is: 0.0
The maximum value of the column MIN_TEMPORAL_GAP is: 1439.0
The mean value of the column MIN_TEMPORAL_GAP is: 95.4028516533885
The median value of the column MIN_TEMPORAL_GAP is: 2.0
The standard deviation of the column MIN_TEMPORAL_GAP is: 174.67081185859826
The 0.05th percentile of the column MIN_TEMPORAL_GAP is: 0.0
The 0.10th percentile of the column MIN_TEMPORAL_GAP is: 0.0
The 25th percentile of the column MIN_TEMPORAL_GAP is: 0.0
The 75th percentile of the column MIN_TEMPORAL_GAP is: 127.0
The 90th percentile of the column MIN_TEMPORAL_GAP is: 382.0
The 95th percentile of the column MIN_TEMPORAL_GAP is: 499.0
The 99th percentile of the column MIN_TEMPORAL_GAP is: 658.0
The 99.9th percentile of the column MIN_TEMPORAL_GAP is: 1052.0
The delta of MIN_TEMPORAL_GAP is: 2.0


In [31]:
# Cleaning operation: remove the rows using the minimum temporal gap

# Save the number of rows before the cleaning operation
shape_before = df.shape[0]

# Delete the rows that have a minimum temporal gap for the same serial and fermata more than the delta calculated before.
# Do not remove the rows with NaN values because they are the first validations of the day of a specific serial and fermata usefull for the analysis
df = df[(df['MIN_TEMPORAL_GAP'] > delta) | (df['MIN_TEMPORAL_GAP'].isna())]

# Print the number of rows before and after the cleaning operation and the difference
print('The number of rows before the cleaning operation is: {}'.format(shape_before))
print('The number of rows after the cleaning operation is: {}'.format(df.shape[0]))
print('The difference is: {}'.format(shape_before - df.shape[0]))
# Calculate the percentage of rows that has just been deleted
print('The percentage of rows that has just been deleted is: {}%'.format(round((shape_before - df.shape[0])/shape_before*100, 2)))

The number of rows before the cleaning operation is: 4876778
The number of rows after the cleaning operation is: 4589440
The difference is: 287338
The percentage of rows that has just been deleted is: 5.89%


In [32]:
# Delete the column MIN_TEMPORAL_GAP because it is not useful anymore
df.drop('MIN_TEMPORAL_GAP', axis=1, inplace=True)

In [33]:
# Create a new dataframe, copied from the original one
df_new = df.copy() 

# Print the head of the new dataframe
print(df_new.head())

# Export the new dataframe in a txt file
# The name of the file is dataset_cleaned followed by the name (file_name variable) of the file that has been cleaned with txt extension
name_file = 'dataset_cleaned_temp' + file_name.split('.')[0] + '.txt'
df_new.to_csv('data/processed/' + name_file, sep='\t', index=False)

print('The script has finished')

        DATA       ORA    DATA_VALIDAZIONE            SERIALE  FERMATA  \
0 2018-09-11  00:01:00 2018-09-11 00:01:00  65676329393943813     1392   
1 2018-09-11  00:01:00 2018-09-11 00:01:00  36412970881802500     5003   
2 2018-09-11  00:01:00 2018-09-11 00:01:00  36412970881806084     5003   
3 2018-09-11  00:01:00 2018-09-11 00:01:00  36412970881805828     5003   
4 2018-09-11  00:01:00 2018-09-11 00:01:00  65676744198028549     5013   

    DESCRIZIONE  TITOLO             DESCRIZIONE_TITOLO  
0  Santa Chiara   12101    Bigl.Aut.75'Mestre/Lido-tsc  
1  Lido (S.M.E.   11109  Biglietto 72 ore Roll. Venice  
2  Lido (S.M.E.   11107        48h-Tpl 24,90-ComVe5,10  
3  Lido (S.M.E.   11107        48h-Tpl 24,90-ComVe5,10  
4  S. Zaccaria    11107        48h-Tpl 24,90-ComVe5,10  
The script has finished


In [38]:
df_cleaned = pd.read_csv("data/processed/dataset_cleaned_tempexport.txt", header=0, sep='\t')
print(df_cleaned.head())
print(df_cleaned.columns)
# Remove the column 'DATA and 'ORA' because they are not useful anymore
df_cleaned.drop(['DATA', 'ORA'], axis=1, inplace=True)
print(df_cleaned.head())
print(df_cleaned.columns)

         DATA       ORA     DATA_VALIDAZIONE            SERIALE  FERMATA  \
0  2018-09-11  00:01:00  2018-09-11 00:01:00  65676329393943813     1392   
1  2018-09-11  00:01:00  2018-09-11 00:01:00  36412970881802500     5003   
2  2018-09-11  00:01:00  2018-09-11 00:01:00  36412970881806084     5003   
3  2018-09-11  00:01:00  2018-09-11 00:01:00  36412970881805828     5003   
4  2018-09-11  00:01:00  2018-09-11 00:01:00  65676744198028549     5013   

    DESCRIZIONE  TITOLO             DESCRIZIONE_TITOLO  
0  Santa Chiara   12101    Bigl.Aut.75'Mestre/Lido-tsc  
1  Lido (S.M.E.   11109  Biglietto 72 ore Roll. Venice  
2  Lido (S.M.E.   11107        48h-Tpl 24,90-ComVe5,10  
3  Lido (S.M.E.   11107        48h-Tpl 24,90-ComVe5,10  
4  S. Zaccaria    11107        48h-Tpl 24,90-ComVe5,10  
Index(['DATA', 'ORA', 'DATA_VALIDAZIONE', 'SERIALE', 'FERMATA', 'DESCRIZIONE',
       'TITOLO', 'DESCRIZIONE_TITOLO'],
      dtype='object')
      DATA_VALIDAZIONE            SERIALE  FERMATA   DESCRIZ

In [39]:
name_file = 'dataset_cleaned_temp' + file_name.split('.')[0] + '.txt'
df_cleaned.to_csv('data/processed/' + name_file, sep='\t', index=False)

print('The script has finished')

The script has finished
